# HeuristicBounds example

This heuristic is implemented after the scipopt primal heuristic 'heur_bounds.h'.
The scipopt library describes this as a "heuristic which fixes all integer variables to a bound (lower/upper) and solves the remaining LP ".

## Installing the packages

In the following we use the `GurobiSolver` and thus need to install gurobi.

In [ ]:
%%capture
!pip install gurobipy

Afterwards, we can install hips.

In [ ]:
%%capture
!pip install https://github.com/cxlvinchau/hips/archive/master.zip

## Example

First add the required imports:

In [ ]:
from hips import *
from hips.heuristics._bounds import HeuristicBounds, BoundDirection
from hips.models import *
from hips.solver import GurobiSolver, ClpSolver

Next we initialize a helper function, which allows us to load a model. The model represents the following MIP:
$$
\begin{array}{lr@{}c@{}r@{}l}
    \text{maximize }   & x_1 + x_2  \\
    \text{subject to } & x_1 + \frac{2}{3} x_2 \leq 2 \\
                       & x_1 + 3 x_2 \leq 3 \\
                       & x_1, x_2 \leq 2\\
                       & x_1, x_2 \geq 0 \\
                       & x_1, x_2 \in \mathbb{Z}
\end{array}
$$

In [ ]:
def build_model(mip_model):
    x = mip_model.add_variable("x", VarTypes.INTEGER, lb=0, ub=2, dim=2)
    constr1 = HIPSArray([1,2/3])*x <= 2
    constr2 = HIPSArray([1,3])*x <= 3
    mip_model.add_constraint(constr1)
    mip_model.add_constraint(constr2)
    obj_func = HIPSArray([1,1])*x
    mip_model.set_objective(obj_func)
    mip_model.lp_model.set_lp_sense(ProblemSense.MAX)

### LOWER

Now we can load this model and use the `HeuristicBounds` heuristic to solve it to the LOWER bound:

In [ ]:
# Test lower bound (LOWER) -> x* = [0,0]
mip_model = MIPModel(GurobiSolver())
build_model(mip_model)
heur = HeuristicBounds(mip_model, BoundDirection.LOWER)
heur.compute()
print("Status: {}".format(heur.get_status()))
print("Found solution: {}".format(heur.get_objective_value()))
print("With Variable values: {}".format({var: heur.variable_solution(var).to_numpy() for var in mip_model.get_variables()}))
print("#---------------------------#")

### UPPER

Fixing the variables to their UPPER bound should yield an infeasible solution:

In [ ]:
#Test upper bound (UPPER) -> Infeasible
mip_model = MIPModel(GurobiSolver())
build_model(mip_model)
heur = HeuristicBounds(mip_model, BoundDirection.UPPER)
heur.compute()
print("Status: {}".format(heur.get_status()))
print("#---------------------------#")

### CLOSEST

At last we solve to the CLOSEST bounds:

In [ ]:
#Test closest bound (CLOSEST) -> [2,0]
mip_model = MIPModel(GurobiSolver())
build_model(mip_model)
heur = HeuristicBounds(mip_model, BoundDirection.CLOSEST)
heur.compute()
print("Status: {}".format(heur.get_status()))
print("Found solution: {}".format(heur.get_objective_value()))
print("With Variable values: {}".format({var: heur.variable_solution(var).to_numpy() for var in mip_model.get_variables()}))
print("#---------------------------#")